# <center>Problem Set 4</center>

## <center> Due March 12 at 10:00 pm

The McHenry County Water Resource Specialist is looking for new ways to view data that is being collected by the county. The USGS is currently collecting data in real-time for a number of wells in McHenry County. Your task in this assignment is to create a potentiometric surface, in real-time, of this data, as well as to look at a surface from summer of 2019 and compare the differences. <b>The wells you will be using for this assignment can be found in the Excel file 'gwmchenry_deep.xlsx'. The files contains columns that (respectively) contain USGS Site Number, Site Name, and Land Surface Elevation.</b>

<i>Note that the code should take into account the possibility for outliers. In particular, think about what happens if your code includes 'NaN' head values (which can and likely will happen at some point over the next two weeks).</i>  

###<center>Part I: Compile Data for Each Well</center>

Using Python, create a Pandas DataFrame that contains the USGS Site Number as the index and columns showing: 
<ul>
    <li>Site Name</li>
    <li>Latitude</li>
    <li>Longitude</li>
    <li>Land Surface Elevation</li>
    <li>Date-time of the most recent data point</li>
    <li>Head (in ft above MSL) of the most recent data point</li>
</ul>

<b>Remember that the USGS web service will return depth to water. You need to convert this to a head.</b>

### <center>Part II: Create maps from this data</center>

Create maps showing the:
<ul>
    <li>Potentiometric surface for Summer 2019 (you may use 200 days prior to the most recent data point)</li>
    <li>Potentiometric surface for today</li>
    <li>Head difference (ft) between the two surfaces</li>
</ul>

Make sure that maps include the following:
<ul>
    <li>State boundaries</li>
    <li>Contours with labels</li>
    <li>A title with the date in a readable format</li>
    <li>A color flood</li>
    <li>A color bar</li>
</ul>

<b>You are responsible to search the internet and determine a  way to assign a color flood and color bar.</b>

### <center>Part III: Comparison with a 2013 study</center>

Conduct a qualitative analysis on the potentiometric surface generated in this assignment with the potentiometric surface generated in a 2013 study (see page 71, Figure 42 of the following report: https://www.ideals.illinois.edu/handle/2142/50040). Write a paragraph discussing these differences (qualitatively). Include an image of Figure 42 in your Jupyter Notebook, with a reference to the report.

In [13]:
import pandas as pd
df = pd.read_excel('https://github.com/dbabrams/Week6PotSurface/blob/master/gwmchenry_deep.xlsx?raw=true', index_col=0)

# df['Latitude']=df['SiteName'].apply(lambda x:x[:4])
# df['Longitude']=df['SiteName'].apply(lambda x:x[4:6])
# df=df.drop(['SiteName'],1)
print(df)
type(df)

                                                   SiteName  elevation
SiteNum                                                               
421056088380801                    43N5E-27.4h1 (14-RIL-S)      806.75
421120088281801                   43N7E-19.8d (HUNT-09-03)      878.00
421122088222702                    43N7E-23.1d2 (16-GRF-D)      879.51
421145088194802                    43N8E-20.6h2 (17-ALG-D)      880.03
421321088341101                   43N6E-07.1g (MARS-09-01)      928.00
421341088283703                    43N6E-01.3b3 (15-COR-D)      851.23
421533088421801   44N5E-30.8c1 (10-MAR-S) (McHenry Co CRN)      780.93
421547088142301                   44N9E-25.1d (WAUC-02-12)      835.00
421626088311401                    44N6E-22.4c1 (11-SEN-I)      830.63
421653088370901                   44N5E-23.5g (MARN-09-02)      827.00
421653088370902                  44N5E-23.5g2 (MARN-10-03)      828.00
421747088270701                  44N7E-17.8h1 (WOOD-08-01)      943.00
421820

pandas.core.frame.DataFrame